# 模型参数的访问、初始化和共享

在[“线性回归的简洁实现”](../chapter_deep-learning-basics/linear-regression-gluon.ipynb)一节中，我们通过`init`模块来初始化模型的全部参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。与之前不同的是，在这里我们使用torch.nn中的`init`模块，它包含了多种模型初始化方法。

In [1]:
import torch
from torch import nn

net = nn.Sequential()
net.add_module("hidden", nn.Linear(20, 256))
net.add_module("activation", nn.ReLU())
net.add_module("output", nn.Linear(256, 10))

X = torch.rand(2, 20)
Y = net(X) # 前向计算

## 访问模型参数

对于使用`Sequential`类构造的神经网络，我们可以通过方括号`[]`来访问网络的任一层。回忆一下上一节中提到的`Sequential`类与`Module`类的继承关系。对于`Sequential`实例中含模型参数的层，我们可以通过`Module`类的`parameters()`或者`named_parameters()`函数来访问该层包含的所有参数。下面，访问多层感知机`net`中隐藏层的所有参数。

In [2]:
for name, param in net.named_parameters():
    print(name, param)
    
type(net.named_parameters())

hidden.weight Parameter containing:
tensor([[-0.0949, -0.0619,  0.0468,  ...,  0.0801,  0.0061,  0.2128],
        [ 0.1147,  0.2220, -0.0054,  ..., -0.0149, -0.1584,  0.1572],
        [ 0.0741, -0.1227, -0.0314,  ...,  0.0975,  0.1925,  0.0592],
        ...,
        [-0.0919, -0.0317, -0.0594,  ..., -0.1490,  0.0025, -0.1462],
        [-0.0113, -0.0613, -0.0177,  ...,  0.0515, -0.0154,  0.2067],
        [-0.0547, -0.0829, -0.1156,  ..., -0.1708, -0.1700,  0.0933]],
       requires_grad=True)
hidden.bias Parameter containing:
tensor([ 1.0377e-01, -3.7003e-02, -8.5052e-02,  2.0793e-01,  1.1988e-01,
        -2.2192e-01,  1.9521e-01, -4.9697e-03,  5.9934e-02, -1.4455e-01,
        -1.6245e-01,  1.8136e-01, -2.1201e-01, -5.1087e-02, -3.1404e-02,
        -6.4125e-02, -1.1353e-01,  9.8943e-02, -1.9422e-01,  8.7757e-02,
        -1.7677e-01,  1.3115e-01, -6.6839e-02, -1.1996e-01,  1.3070e-01,
        -1.3522e-01,  2.2208e-01, -6.1894e-02,  6.3665e-02, -1.4802e-01,
        -1.5915e-02, -2.2043e-0

generator

可以看到，我们得到了一个返回参数名和参数值的迭代器。其中隐藏层权重参数的名称为`hidden.weight`，它由`net[0]`的名称（`hidden`）和自己的变量名（`weight`）组成。


*注：如果单独调用`net[0].named_parameters()`获得的权重参数名为`weight`*


为了访问特定参数，我们可以使用`net.state_dict()`来获得一个由参数名映射到参数值的字典（类型为`OrderedDict`）。通过名字来访问字典里的元素，也可以直接使用它的变量名。下面两种方法是等价的，但通常后者的代码可读性更好。

In [3]:
net[0].state_dict()['weight'], net[0].weight

(tensor([[-0.0949, -0.0619,  0.0468,  ...,  0.0801,  0.0061,  0.2128],
         [ 0.1147,  0.2220, -0.0054,  ..., -0.0149, -0.1584,  0.1572],
         [ 0.0741, -0.1227, -0.0314,  ...,  0.0975,  0.1925,  0.0592],
         ...,
         [-0.0919, -0.0317, -0.0594,  ..., -0.1490,  0.0025, -0.1462],
         [-0.0113, -0.0613, -0.0177,  ...,  0.0515, -0.0154,  0.2067],
         [-0.0547, -0.0829, -0.1156,  ..., -0.1708, -0.1700,  0.0933]]),
 Parameter containing:
 tensor([[-0.0949, -0.0619,  0.0468,  ...,  0.0801,  0.0061,  0.2128],
         [ 0.1147,  0.2220, -0.0054,  ..., -0.0149, -0.1584,  0.1572],
         [ 0.0741, -0.1227, -0.0314,  ...,  0.0975,  0.1925,  0.0592],
         ...,
         [-0.0919, -0.0317, -0.0594,  ..., -0.1490,  0.0025, -0.1462],
         [-0.0113, -0.0613, -0.0177,  ...,  0.0515, -0.0154,  0.2067],
         [-0.0547, -0.0829, -0.1156,  ..., -0.1708, -0.1700,  0.0933]],
        requires_grad=True))

权重梯度的形状和权重的形状一样。因为我们还没有进行反向传播计算，所以梯度为None。

In [4]:
net[0].weight.grad == None

True

类似地，我们可以访问其他层的参数，如输出层的偏差值。

In [5]:
net[2].bias

Parameter containing:
tensor([-0.0379, -0.0311,  0.0163,  0.0259,  0.0206, -0.0063, -0.0559, -0.0193,
         0.0398,  0.0052], requires_grad=True)

## 初始化模型参数

我们在[“数值稳定性和模型初始化”](../chapter_deep-learning-basics/numerical-stability-and-init.ipynb)一节中描述了模型的默认初始化方法：权重参数元素为[-0.07, 0.07]之间均匀分布的随机数，偏差参数则全为0。但我们经常需要使用其他方法来初始化权重。PyTorch在`nn.init`模块里提供了多种预设的初始化单个参数的方法。在下面的例子中，我们将隐藏层的权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并将偏差参数初始化为常数0。

In [6]:
nn.init.normal_(net[0].weight, mean=0, std=0.01)  # weight
nn.init.constant_(net[0].bias, 0) # bias

net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 8.1788e-03, -2.2569e-05,  1.8563e-02,  4.3964e-04, -6.7969e-03,
         -2.7120e-02, -1.8878e-02, -6.7642e-03, -1.7379e-03, -5.9111e-03,
          6.1728e-03,  2.4607e-03,  1.6794e-02,  4.0205e-03,  2.8080e-02,
          2.5098e-03, -1.9734e-02,  1.0123e-02,  1.6951e-02, -1.0615e-02]),
 tensor(0.))

## 自定义初始化方法

如果需要将模型中所有网络层的参数按照相同的策略进行初始化，可以自定义一个初始化方法（如`weight_init`），然后使用`.apply(weight_init)`进行自定义初始化。在下面的例子里，我们令Linear层的权重有一半概率初始化为0，有另一半概率初始化为$[-10,-5]$和$[5,10]$两个区间里均匀分布的随机数。


参考自：<https://discuss.pytorch.org/t/reset-the-parameters-of-a-model/29839>

In [7]:
# PyTorch不允许对requires_gead=True的tensor做inplace操作。
# 所以需要取出weight的 data 属性，对它进行相应的处理
def weight_init(m):
    if isinstance(m, nn.Linear):
        print('Init', m.weight.shape)
        m.weight.data.uniform_(-10, to=10)
        m.weight.data *= (m.weight.data.abs() >= 5).float()

net.apply(weight_init)
net[0].weight.data[0]

Init torch.Size([256, 20])
Init torch.Size([10, 256])


tensor([-0.0000, -0.0000,  0.0000, -8.4261,  8.0859,  8.2111, -0.0000, -9.6193,
        -6.4062, -0.0000,  7.2762,  0.0000, -0.0000,  9.1960,  0.0000,  0.0000,
         0.0000, -5.0190,  0.0000,  9.5519])

*注：因为 PyTorch 并没有提供对模型所有参数进行初始化的方法，我们写一个自定义的全局初始化参数的方法加入到 d2ltorch 包中。(该方法来自 [Weights-Initializer-pytorch](https://github.com/3ammor/Weights-Initializer-pytorch) )*

In [8]:
def params_init(model, init, **kwargs): # 本方法已保存在d2ltorch包中方便以后使用

    def initialize(m):
        if isinstance(m, nn.Conv2d):
            init(m.weight.data, **kwargs)
            try:
                init(m.bias.data)
            except:
                pass

        elif isinstance(m, nn.Linear):
            init(m.weight.data, **kwargs)
            try:
                init(m.bias.data)
            except:
                pass

        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1.0)
            m.bias.data.fill_(0)

        elif isinstance(m, nn.BatchNorm1d):
            m.weight.data.fill_(1.0)
            m.bias.data.fill_(0)

    model.apply(initialize)

此外，我们还可以通过`Parameter`类的`data`属性来直接改写模型参数。例如，在下例中我们将隐藏层参数在现有的基础上加1。

In [9]:
net[0].weight.data = net[0].weight.data + 1
net[0].weight.data[0]

tensor([ 1.0000,  1.0000,  1.0000, -7.4261,  9.0859,  9.2111,  1.0000, -8.6193,
        -5.4062,  1.0000,  8.2762,  1.0000,  1.0000, 10.1960,  1.0000,  1.0000,
         1.0000, -4.0190,  1.0000, 10.5519])

## 共享模型参数

在有些情况下，我们希望在多个层之间共享模型参数。[“模型构造”](model-construction.ipynb)一节介绍了如何在`Module`类的`forward`函数里多次调用同一个层来计算。

*注：经过查找，PyTorch并没有提供其他的共享参数并且传播时保持梯度的方法。*

## 小结

* 有多种方法来访问、初始化和共享模型参数。
* 可以自定义初始化方法。


## 练习

* 查阅有关`nn.init`模块的PyTorch文档，了解不同的参数初始化方法。
* 尝试在`net`实例化后、`net(X)`前访问模型参数，观察模型参数的形状。
* 构造一个含共享参数层的多层感知机并训练。在训练过程中，观察每一层的模型参数和梯度。



## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/987)

![](../img/qr_parameters.svg)